## Erster Test des lokalen LLM

In [2]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from final_code.pubmed_api import pubmed_api_pull
import requests


In [9]:
def ask_mistral(prompt):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "mistral",  # Name des installierten Modells
            "prompt": prompt,
            "stream": False
        }
    )
    return response.json()["response"]

# Beispiel:
antwort = ask_mistral("Wie alt ist die Erde?")
# Ausgabe der Antwort
print(antwort)

 Die Erde ist ungefähr 4,6 Milliarden Jahre alt. Diese Zahl basiert auf dem uran-lead-Alter des Hedeynit-Meteoriten, der von der Erde stammen soll und ursprünglich zu den Asteroiden gehörte. Dies ist ein gebräuchliches Datum zur Abschätzung des Alters unseres Planeten. Es gibt jedoch alternative Theorien über das Alter der Erde, die bis zu einer Alter von 10 Milliarden Jahre reichen.


## Code mit Connection-Check

In [ ]:
import requests

def check_connection():
    """Prüft ob Ollama-Server läuft (Timeout 3 Sekunden)"""
    try:
        return requests.get("http://localhost:11434/api/tags", timeout=3).ok
    except:
        return False

def ask_mistral(prompt):
    if not check_connection():
        raise ConnectionError("Ollama-Server nicht erreichbar (localhost:11434)")
    
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "mistral",
            "prompt": prompt,
            "stream": False
        }
    )
    return response.json()["response"]

# Beispiel mit integriertem Check:
if check_connection():
    print(ask_mistral("What is avian influenza?"))
else:
    print("Fehler: Ollama nicht gestartet oder Port 11434 blockiert")

In [ ]:
search = ["avian influenza outbreak"]
no = 20

df = pubmed_api_pull(search, no)

In [14]:
text = df.loc[1,'full_text']
print(text)

1 Public health strategies for avian influenza mitigation To effectively mitigate this public health threat, enhanced surveillance of poultry and human populations is critical for early detection and containment of outbreaks, including. • Wastewater Surveillance: Employed in the United States to detect avian influenza A(H5N1) virus by analysing wastewater samples for viral RNA, facilitating early outbreak detection [ 5 ]. • Vaccination of Poultry: Implemented globally to reduce AI transmission among birds and minimise interspecies spread
2 Strengthening cross-border cooperation Cross-border collaboration with Pakistan, Bangladesh, and Nepal can facilitate the exchange of surveillance data, harmonise control measures, and manage poultry trade to prevent the spread of avian influenza.
3 Public awareness initiatives • Community Workshops: Educating poultry farmers and handlers on avian influenza symptoms and biosecurity measures. • Mass Media Campaigns: Using television, radio, and social

In [10]:
prompt = """Du bist ein KI-gestütztes Analysetool, das wissenschaftliche Texte untersucht, um Informationen über Krankheitsausbrüche zu extrahieren. Deine Aufgabe ist es, relevante Informationen aus dem bereitgestellten Text präzise und strukturiert wiederzugeben.

User Prompt:
Lies den folgenden Text und beantworte die Fragen:

    Gab es einen Ausbruch dieser Krankheit? (Ja/Nein)

    Falls ja, wo fand der Ausbruch statt? (Ort oder Region angeben)

    Wie viele Menschen waren betroffen? (Anzahl der Fälle, falls vorhanden)

    Wann war der Ausbruch? (Datum oder Zeitraum)

Falls eine Information nicht im Text vorhanden ist, gib „Nicht angegeben“ aus. Hier der Text: """ + text + """Antwort:

    Gab es einen Ausbruch? → [...]

    Wo? → [...]

    Wie viele Leute waren betroffen? → [...]

    Wann? → [...]"""

antwort = ask_mistral(prompt)
print(antwort)

1. Was gab es ausgebrochen? (Was was there an outbreak of?) - There was no specific outbreak mentioned in the text, but it refers to Avian Influenza A(H5N1).

2. Where? (Wo?) - The study was conducted among individuals in contact with domestic and/or wild birds in the UK.

3. How many people were affected? (Wie viele Leute waren betroffen?) - No specific number of people affected by the outbreak was mentioned in the text. However, it mentions that data collected will be used to describe the contact patterns of individuals in contact with birds, which can help inform mathematical models of transmission.

4. When? (Wann?) - The study was conducted between May and July 2024.


In [ ]:
def ask_mistral(prompt):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "llama3.2",  # Name des installierten Modells
            "prompt": prompt,
            "stream": False
        }
    )
    return response.json()["response"]

antwort = ask_mistral(prompt)
print(antwort)

In [16]:
text = "there was an outbreak on july 1 in london. 20 People affected"
text2 = "BLALALALALALA"


def check_if_text_has_outbreak(text):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "llama3.2",
            "prompt": 
                """You will receive a text below. 
                I want you to analyze whether a disease outbreak has occurred. If an outbreak is detected, return 'True'; otherwise, return 'False'. 
                Here is the text:""" + text,
            "stream": False
        }
    )
    return response.json()["response"]

def extract_outbreak_place(text):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "mistral",
            "prompt": 
                """Im folgenden erhältst du einen Text. Bitte untersuche diesen Text daraufhin an welchem Ort es einen Ausbruch
                 der Krankheit gegeben hat. Gib bitte ausschließlich den Ort zurück. Wenn es mehrere Ort gibt, gib sie mit
                  einem Semikolon getrennt an. Wenn kein Ort spezifiziert ist gib bitte N/A zurück. Hier der Text: """ + text,
            "stream": False
        }
    )
    return response.json()["response"]

def extract_outbreak_time(text):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "mistral",
            "prompt": 
                """Im folgenden erhältst du einen Text. Bitte untersuche diesen Text daraufhin zu welchem Zeitpunkt es einen Ausbruch
                 der Krankheit gegeben hat. Zeitpunkte können bestimmte Tage, bestimmte Monage oder Wochen sein. Gib bitte immer den Monat oder
                 die Woche zusammen mit dem Jahr (falls vorhanden)
                 Gib bitte ausschließlich den Zeitpunkt zurück. Wenn es mehrere Zeitpunkte gibt, gib sie mit
                  einem Semikolon getrennt an. Wenn kein Zeitpunkt spezifiziert ist gib bitte N/A zurück. Hier der Text: """ + text,
            "stream": False
        }
    )
    return response.json()["response"]


answer = check_if_text_has_outbreak(text2)
print(answer)

Based on the input, I cannot conclude that a disease outbreak has occurred. The text appears to be nonsensical and does not contain any relevant information about a disease outbreak.

Therefore, I will return 'False'.
